In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import glob
%matplotlib inline

# import data

In [ ]:
train_img = glob.glob(r'C:\Users\User\Downloads\kaggle\image_seperate_test\dataset1\*.png')
mask_img = glob.glob(r'C:\Users\User\Downloads\kaggle\image_seperate_test\Labeled\*.png')
df = pd.DataFrame([]);
df['train_img'] = train_img;df['mask_img'] = mask_img

In [ ]:
def read_train(url):
    img = cv2.imread(url,0)
    img = cv2.resize(img,(128,128))
    img = img.reshape(128,128,1)
    return img

In [ ]:
def read_mask(url):
    img = cv2.imread(url,0)
    img = cv2.resize(img,(128,128))
    _,img = cv2.threshold(img,100,255,cv2.THRESH_BINARY)
    img = img.reshape(128,128,1)
    return img

In [ ]:
cv2.imshow('img',img_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
train = np.array([read_train(df['train_img'][i]) for i in range(len(df))])
mask = np.array([read_mask(df['mask_img'][i]) for i in range(len(df))])

In [ ]:
train = train/255
mask  = mask /255

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train,mask,test_size=0.2,random_state=0)

# create model

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dropout,Conv2DTranspose,Input,Activation,concatenate

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [ ]:
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
input_img = Input((128, 128, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.3, batchnorm=True)
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
results = model.fit(x_train, y_train, batch_size=30, epochs=50,
                    validation_data=(x_test, y_test))

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
sample = np.random.randint(0,50,1)
plt.figure(figsize=(16,8))
plt.subplot(2,3,1)
plt.imshow(y_pred[sample].reshape(128,128)*256)
plt.subplot(2,3,2)
plt.imshow(y_test[sample].reshape(128,128)*256)
plt.subplot(2,3,3)
plt.imshow(x_test[sample].reshape(128,128)*256)
plt.subplot(2,3,4)
plt.imshow(y_pred[sample+1].reshape(128,128)*256)
plt.subplot(2,3,5)
plt.imshow(y_test[sample+1].reshape(128,128)*256)
plt.subplot(2,3,6)
plt.imshow(x_test[sample+1].reshape(128,128)*256)